In [ ]:
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn
import torch
import math
import copy

In [ ]:
img_dim = 32

data_transforms = {
    "treino": transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((img_dim, img_dim)),
        transforms.ToTensor()
    ]),
    "validacao": transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((img_dim, img_dim)),
        transforms.ToTensor()
    ]),
    "teste": transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((img_dim, img_dim)),
        transforms.ToTensor()
    ]),
}

# Processo de saparacao feito direto nos folders
sets = ["treino", "validacao", "teste"]
data_dir = "/content/datasets"
image_datasets = {x: datasets.ImageFolder(root=f'{data_dir}/{x}', transform=data_transforms[x]) for x in sets}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=2) for x in sets}

dataset_sizes = {x: len(image_datasets[x]) for x in sets}
class_names = image_datasets['treino'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Conectado em um ambiente com cuda:0


In [ ]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(input_size, hidden_sizes[0]),
            nn.ReLU(),
            nn.Linear(hidden_sizes[0], hidden_sizes[1]),
            nn.ReLU(),
            nn.Linear(hidden_sizes[1], hidden_sizes[2]),
            nn.ReLU(),
            nn.Linear(hidden_sizes[2], output_size)
        )


    def forward(self, x):
        return self.layers(x)

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, max_epochs=10, grace_period = 3):
    best_loss = math.inf
    curr_grace_period = 0
    best_model = copy.deepcopy(model.state_dict())

    for epoch in range(max_epochs):
        print(f"Época {epoch+1}/{max_epochs}")
        print('-' * 10)

        for phase in ["treino", "validacao"]:
            if phase == "treino":
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0
            for inputs, labels in dataloaders[phase]:
                optimizer.zero_grad()

                inputs, labels = inputs.to(device), labels.to(device)
                with torch.set_grad_enabled(phase == "treino"):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == "treino":
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}', end="\n")

            if phase == "validacao":
              if epoch_loss < best_loss:
                best_loss = epoch_loss
                curr_grace_period = 0
                best_model = copy.deepcopy(model.state_dict())
              else:
                curr_grace_period += 1
                if curr_grace_period >= grace_period:
                  print("Early stopping")
                  model.load_state_dict(best_model)
                  return

    model.load_state_dict(best_model)
    return

In [ ]:
input_size = img_dim * img_dim 
hidden_sizes = [512, 256, 128]
output_size = len(class_names)

model_ft = MLP(input_size, hidden_sizes, output_size)
model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

print("Treinando")
train_model(model_ft, dataloaders, criterion, optimizer_ft, max_epochs=20, grace_period=5)

print("Avaliando no conjunto de testes")
model_ft.eval()
test_corrects = 0

for inputs, labels in dataloaders['teste']:
    inputs = inputs.to(device)
    labels = labels.to(device)

    outputs = model_ft(inputs)
    _, preds = torch.max(outputs, 1)
    test_corrects += torch.sum(preds == labels.data)

test_acc = test_corrects.double() / dataset_sizes['teste']
print(f'Test Acc: {test_acc:.4f}')

Treinando
Época 1/20
----------
treino Loss: 0.6466 Acc: 0.6425

validacao Loss: 0.6076 Acc: 0.6561

Época 2/20
----------
treino Loss: 0.3287 Acc: 0.9069

validacao Loss: 0.5300 Acc: 0.6245

Época 3/20
----------
treino Loss: 0.1188 Acc: 0.9691

validacao Loss: 0.4497 Acc: 0.7491

Época 4/20
----------
treino Loss: 0.0583 Acc: 0.9837

validacao Loss: 0.5218 Acc: 0.7366

Época 5/20
----------
treino Loss: 0.0333 Acc: 0.9909

validacao Loss: 0.5220 Acc: 0.7601

Época 6/20
----------
treino Loss: 0.0255 Acc: 0.9934

validacao Loss: 0.3863 Acc: 0.8440

Época 7/20
----------
treino Loss: 0.0196 Acc: 0.9950

validacao Loss: 0.5636 Acc: 0.7549

Época 8/20
----------
treino Loss: 0.0168 Acc: 0.9957

validacao Loss: 0.4121 Acc: 0.8226

Época 9/20
----------
treino Loss: 0.0158 Acc: 0.9955

validacao Loss: 0.4813 Acc: 0.8031

Época 10/20
----------
treino Loss: 0.0124 Acc: 0.9968

validacao Loss: 0.4202 Acc: 0.8370

Época 11/20
----------
treino Loss: 0.0132 Acc: 0.9961

validacao Loss: 0.3322 